# Practice Random Forest Pipeline/Workflow

In [189]:
# Imports
import pandas as pd
import numpy as np
import random
import string
from timeit import default_timer as timer # Time how long commands take

import warnings
warnings.filterwarnings('ignore') # Ignore warnings to clean up visual output change back to 'default' to see them


### Create fake/dummy data

In [190]:
row_num = 5000

In [191]:
# Create columns names for dataset
columns = [''.join(random.choices(string.ascii_uppercase + string.digits, k=5)) for x in range(380)]

In [192]:
# Add Thyroid cancer to columns
columns.insert(0, 'userId')
# Add userId to columns
columns.append('thyroid_cancer')

In [193]:
# Create dataframe rows with different possible data types

dictionary = {}

for item in columns:

    column_options = [[random.randint(0,1) for i in range(row_num)], np.random.randint(0, 10, row_num, dtype=int), np.random.randint(0, 3, row_num, dtype=int),
                      np.random.random_sample(size = row_num), np.random.uniform(low=0, high=3000, size=row_num), np.random.uniform(low=0, high=150, size=row_num)]

    dictionary[item] = random.choice(column_options)


In [194]:
# Create the pandas DataFrame
df = pd.DataFrame(dictionary)

In [195]:
# Change userId values
df['userId'] = [x for x in range(row_num)]

# Make userId index
df.set_index('userId', inplace=True)

In [196]:
# Add missing values
for col in df.columns:
    df.loc[df.sample(frac=0.1).index, col] = np.nan

In [197]:
# Make thyroid cancer data binary with no missing values
df['thyroid_cancer'] = [random.randint(0, 1) for x in range(row_num)]

### Data preprocessing

In [198]:
df


,QI8KX,EW5XJ,394MJ,5JCY6,N2TG9,MWSMR,M57ER,U0DHH,RWCWO,MAR0D,...,W18UI,ANLUA,66BMD,DKW6X,UB65D,G1106,Z55FK,5LZSG,0XI7C,thyroid_cancer
userId,,,,,,,,,,,,,,,,,,,,,
0,2.0,178.411894,64.271156,7.0,7.0,0.342611,79.245015,0.0,1.0,1.0,...,6.0,4.0,2645.203075,54.583515,2764.888478,0.0,138.187447,1.0,2285.403528,0
1,2.0,41.058191,67.988225,0.0,3.0,0.504474,NaN,1.0,0.0,2.0,...,8.0,7.0,180.707016,41.534475,2620.283367,1.0,37.911595,0.0,2262.750325,1
2,1.0,1118.457041,147.822393,0.0,NaN,0.780437,134.648108,1.0,1.0,2.0,...,NaN,1.0,2083.378063,105.312080,501.798643,1.0,128.040195,0.0,1650.003516,0
3,1.0,2690.553815,103.377883,3.0,3.0,0.291342,145.199941,2.0,1.0,0.0,...,9.0,NaN,2405.162930,NaN,2435.913390,1.0,11.145034,0.0,NaN,1
4,0.0,615.266363,49.694446,1.0,4.0,0.123044,64.328732,0.0,1.0,0.0,...,9.0,9.0,1506.176535,88.040958,1165.386309,0.0,74.357761,1.0,1292.899258,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,2.0,364.680844,NaN,2.0,0.0,0.032761,38.924141,2.0,0.0,0.0,...,2.0,1.0,430.500069,107.329729,214.951819,0.0,127.465821,1.0,2432.780735,1
4996,1.0,NaN,1.225612,2.0,2.0,0.886880,110.135788,2.0,1.0,0.0,...,9.0,6.0,59.799045,NaN,1397.923996,1.0,62.060686,0.0,2388.699113,1
4997,0.0,1689.256828,NaN,6.0,0.0,0.019575,13.579529,1.0,1.0,2.0,...,3.0,8.0,301.623017,73.243115,2118.443840,1.0,98.669090,0.0,650.083062,0


In [199]:
# Make sure thyroid cancer column has been added

In [200]:
# Make sure diseases descriptions are the column names rather than just phecodes

In [201]:
# Manual feature selection - use mutual information and consider data leakage.

### Pipeline

In [202]:
# Create test-train split
df

,QI8KX,EW5XJ,394MJ,5JCY6,N2TG9,MWSMR,M57ER,U0DHH,RWCWO,MAR0D,...,W18UI,ANLUA,66BMD,DKW6X,UB65D,G1106,Z55FK,5LZSG,0XI7C,thyroid_cancer
userId,,,,,,,,,,,,,,,,,,,,,
0,2.0,178.411894,64.271156,7.0,7.0,0.342611,79.245015,0.0,1.0,1.0,...,6.0,4.0,2645.203075,54.583515,2764.888478,0.0,138.187447,1.0,2285.403528,0
1,2.0,41.058191,67.988225,0.0,3.0,0.504474,NaN,1.0,0.0,2.0,...,8.0,7.0,180.707016,41.534475,2620.283367,1.0,37.911595,0.0,2262.750325,1
2,1.0,1118.457041,147.822393,0.0,NaN,0.780437,134.648108,1.0,1.0,2.0,...,NaN,1.0,2083.378063,105.312080,501.798643,1.0,128.040195,0.0,1650.003516,0
3,1.0,2690.553815,103.377883,3.0,3.0,0.291342,145.199941,2.0,1.0,0.0,...,9.0,NaN,2405.162930,NaN,2435.913390,1.0,11.145034,0.0,NaN,1
4,0.0,615.266363,49.694446,1.0,4.0,0.123044,64.328732,0.0,1.0,0.0,...,9.0,9.0,1506.176535,88.040958,1165.386309,0.0,74.357761,1.0,1292.899258,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,2.0,364.680844,NaN,2.0,0.0,0.032761,38.924141,2.0,0.0,0.0,...,2.0,1.0,430.500069,107.329729,214.951819,0.0,127.465821,1.0,2432.780735,1
4996,1.0,NaN,1.225612,2.0,2.0,0.886880,110.135788,2.0,1.0,0.0,...,9.0,6.0,59.799045,NaN,1397.923996,1.0,62.060686,0.0,2388.699113,1
4997,0.0,1689.256828,NaN,6.0,0.0,0.019575,13.579529,1.0,1.0,2.0,...,3.0,8.0,301.623017,73.243115,2118.443840,1.0,98.669090,0.0,650.083062,0


# One Hot Encoding

In [203]:
# Find categorical data columns in dataframe
def find_categorical_columns(dataframe):

    columns = [list(dataframe[i]) for i in dataframe]

    uniques = [len(set([i for i in a if pd.notna(i)])) for a in columns] # Num of unique values in a column

    categorical_indexes = [i for i, c in enumerate(uniques) if c <= 10]

    unique_cat_num = [uniques[i] for i in categorical_indexes]

    index_ncats = list(zip(categorical_indexes, unique_cat_num))

    return index_ncats # return indexes of categorical data and the number of categories for that index

# Remove binary columns from categorical column list
def remove_binary(zipped_list):

    new_list = [list(item) for item in zipped_list if item[1] != 2]

    return new_list # Index first, cat num second


In [204]:
# Find categorical columns
zl = find_categorical_columns(df)
# Remove binary categorical columns
cats = remove_binary(zl)
# List of categorical column indexes
one_hot_col_indexes = [i[0] for i in cats]
# List of column names
cat_cols = [df.columns[x] for x in one_hot_col_indexes]

In [205]:
from sklearn.preprocessing import OneHotEncoder

# create instance of one hot encoder
enc = OneHotEncoder()

In [206]:
# One hot encode the data
enc_data = enc.fit_transform(df[cat_cols]).toarray()
# New column names
enc_feat = enc.get_feature_names_out(cat_cols)

In [207]:
enc_feat

array(['QI8KX_0.0', 'QI8KX_1.0', 'QI8KX_2.0', ..., 'G1106_1.0',
       'G1106_2.0', 'G1106_nan'], dtype=object)

In [208]:
enc_data

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 1., 0., 0.],
       [0., 1., 0., ..., 1., 0., 0.],
       ...,
       [1., 0., 0., ..., 1., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]])

In [209]:
encoded_df = pd.DataFrame(enc_data, columns=enc_feat)

In [210]:
encoded_df

,QI8KX_0.0,QI8KX_1.0,QI8KX_2.0,QI8KX_nan,5JCY6_0.0,5JCY6_1.0,5JCY6_2.0,5JCY6_3.0,5JCY6_4.0,5JCY6_5.0,...,ANLUA_5.0,ANLUA_6.0,ANLUA_7.0,ANLUA_8.0,ANLUA_9.0,ANLUA_nan,G1106_0.0,G1106_1.0,G1106_2.0,G1106_nan
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4996,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4997,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
4998,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


# Imputation

Selecting MissForest Random Forest imputation as it has been described the most successful algorithm (like in this [paper](https://www.frontiersin.org/articles/10.3389/fdata.2021.693674/full)), its relatively low level of computational 'greedyness', and due to the fact it is nonparametric. Another description of how it works can also be found [here](https://betterdatascience.com/python-missforest-algorithm/).

See the [documentation](https://github.com/epsilon-machine/missingpy) on github for further details on MissForest.

A basic example of how to use it can be found [here](https://betterdatascience.com/python-missforest-algorithm/).

In [211]:
# Import required for missforest due to new sklearn version renaming
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
# Import package for MissForest
from missingpy import MissForest

In [212]:
# Initialise imputer
imputer = MissForest(max_iter=5)

# Test imputation on subset of dataframe
s = timer()
imputed = imputer.fit_transform(df[df.columns[0:100]], cat_vars=[x for x in one_hot_col_indexes if x < 100]) # Test on subset of dataframe
e = timer()
print(f'Imputation time: {(e - s)/60} mins')

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Imputation time: 16.518515841666765 mins


In [213]:
# Imputed data frame
imputed_df = pd.DataFrame(imputed, columns=df.columns[0:100])
imputed_df

,QI8KX,EW5XJ,394MJ,5JCY6,N2TG9,MWSMR,M57ER,U0DHH,RWCWO,MAR0D,...,Y3K0J,P97EB,6IAMA,FQ2EP,PT1HK,AYUOY,WBN7F,HG7C0,TUQX1,B1OIL
0,2.0,178.411894,64.271156,7.0,7.0,0.342611,79.245015,0.0,1.0,1.0,...,6.0,0.996116,2.0,9.397782,2.0,0.0,0.498976,1.0,2.0,103.285235
1,2.0,41.058191,67.988225,0.0,3.0,0.504474,79.834724,1.0,0.0,2.0,...,7.0,0.543821,0.0,93.616651,0.0,1.0,0.589777,1.0,0.0,21.555428
2,1.0,1118.457041,147.822393,0.0,3.0,0.780437,134.648108,1.0,1.0,2.0,...,2.0,0.494028,1.0,14.240045,0.0,1.0,0.572293,1.0,1.0,96.054583
3,1.0,2690.553815,103.377883,3.0,3.0,0.291342,145.199941,2.0,1.0,0.0,...,4.0,0.001430,0.0,11.934569,1.0,2.0,0.723056,0.0,1.0,53.042763
4,0.0,615.266363,49.694446,1.0,4.0,0.123044,64.328732,0.0,1.0,0.0,...,1.0,0.684642,1.0,131.100466,2.0,0.0,0.455917,0.0,1.0,104.397814
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,2.0,364.680844,78.646507,2.0,0.0,0.032761,38.924141,2.0,0.0,0.0,...,7.0,0.750833,2.0,140.076844,2.0,0.0,0.725573,0.0,2.0,0.668651
4996,1.0,1613.558176,1.225612,2.0,2.0,0.886880,110.135788,2.0,1.0,0.0,...,7.0,0.297160,1.0,82.222408,1.0,2.0,0.673923,1.0,2.0,44.011999
4997,0.0,1689.256828,74.730057,6.0,0.0,0.019575,13.579529,1.0,1.0,2.0,...,2.0,0.596576,2.0,86.134989,2.0,0.0,0.661564,1.0,0.0,22.645691
4998,1.0,1469.869266,14.525267,8.0,5.0,0.057964,79.890496,2.0,0.0,2.0,...,9.0,0.528380,1.0,145.756246,0.0,1.0,0.553282,0.0,0.0,27.741351


In [214]:
df[df.columns[0:100]]

,QI8KX,EW5XJ,394MJ,5JCY6,N2TG9,MWSMR,M57ER,U0DHH,RWCWO,MAR0D,...,Y3K0J,P97EB,6IAMA,FQ2EP,PT1HK,AYUOY,WBN7F,HG7C0,TUQX1,B1OIL
userId,,,,,,,,,,,,,,,,,,,,,
0,2.0,178.411894,64.271156,7.0,7.0,0.342611,79.245015,0.0,1.0,1.0,...,6.0,0.996116,2.0,9.397782,2.0,0.0,NaN,1.0,NaN,103.285235
1,2.0,41.058191,67.988225,0.0,3.0,0.504474,NaN,1.0,0.0,2.0,...,7.0,0.543821,0.0,93.616651,NaN,1.0,0.589777,1.0,0.0,21.555428
2,1.0,1118.457041,147.822393,0.0,NaN,0.780437,134.648108,1.0,1.0,2.0,...,2.0,0.494028,1.0,14.240045,0.0,1.0,0.572293,1.0,1.0,96.054583
3,1.0,2690.553815,103.377883,3.0,3.0,0.291342,145.199941,2.0,1.0,0.0,...,4.0,0.001430,0.0,11.934569,1.0,2.0,0.723056,0.0,1.0,53.042763
4,0.0,615.266363,49.694446,1.0,4.0,0.123044,64.328732,0.0,1.0,0.0,...,1.0,0.684642,1.0,131.100466,2.0,0.0,0.455917,0.0,1.0,104.397814
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,2.0,364.680844,NaN,2.0,0.0,0.032761,38.924141,2.0,0.0,0.0,...,NaN,0.750833,2.0,140.076844,2.0,0.0,0.725573,0.0,2.0,0.668651
4996,1.0,NaN,1.225612,2.0,2.0,0.886880,110.135788,2.0,1.0,0.0,...,NaN,0.297160,1.0,82.222408,1.0,2.0,0.673923,1.0,2.0,44.011999
4997,0.0,1689.256828,NaN,6.0,0.0,0.019575,13.579529,1.0,1.0,2.0,...,2.0,0.596576,2.0,86.134989,2.0,NaN,0.661564,1.0,0.0,22.645691


# Feature Scaling

A brief article detailing information about feature scaling can be found [here](https://towardsdatascience.com/all-about-feature-scaling-bcc0ad75cb35) and [here](https://www.analyticsvidhya.com/blog/2020/04/feature-scaling-machine-learning-normalization-standardization/).

Two common methods are normalisation and standardisation. [Normalisation](https://www.geeksforgeeks.org/what-is-data-normalization/) involves bounding our numbers between two values such as 0 and 1. [Standardisation](https://www.sisense.com/glossary/data-standardization/) transforms the data to have zero mean and a variance of 1, making the data unitless.

As stated in [this](https://www.geeksforgeeks.org/normalization-vs-standardization/) GeeksforGeeks article, normalisation struggles with outliers. For example, it would be better suited to age data rather than salary data. Standardisation is better suited to data with a Gaussian/normal distribution and is relatively unaffected by outliers.


In [216]:
# Get the continuous numerical columns in a dataframe
def find_continuous_columns(dataframe):

    columns = [list(dataframe[i]) for i in dataframe]

    uniques = [len(set([i for i in a if pd.notna(i)])) for a in columns] # Num of unique values in a column

    continuous_indexes = [i for i, c in enumerate(uniques) if c > 10] # Indexes of continuous columns

    # List of column names
    con_cols = [dataframe.columns[x] for x in continuous_indexes]

    return con_cols # return list of continuous column names

In [217]:
# Scale the data
from sklearn.preprocessing import MinMaxScaler, StandardScaler

def sk_scaling(df, scaling='s'):

    if scaling == 's':
        scaler = StandardScaler()
        scaled = scaler.fit_transform(df)
        return scaled

    elif scaling == 'n':
        scaler = MinMaxScaler()
        scaled = scaler.fit_transform(df)
        return scaled

    else:

        print(f'Incorrect scaling parameter')


In [218]:
# List of column names for continuous columns
continuous_df = df[find_continuous_columns(df)]

In [219]:
# Apply Normalised feature scaling
scaled_df = sk_scaling(continuous_df, 'n')

In [220]:
scaled_df

array([[0.05944203, 0.42815548, 0.34261273, ..., 0.92193613, 0.9213182 ,
        0.76172814],
       [0.01364084, 0.45295483, 0.50456458, ..., 0.87370724, 0.25267846,
        0.75417266],
       [0.3729042 , 0.98558849, 0.78067878, ..., 0.16714741, 0.85365628,
        0.54980464],
       ...,
       [0.56323988,        nan, 0.01939916, ..., 0.70633306, 0.65780965,
        0.21630349],
       [0.49008414, 0.09626332, 0.05780914, ...,        nan, 0.04249639,
        0.62147281],
       [0.66322748, 0.68046087, 0.1058661 , ..., 0.34451309, 0.81382018,
        0.69929065]])

# Oversampling or Undersampling

Oversampling and undersampling are methods applied to an imbalanced dataset where there is a risk of the minority class being ignored by ML algorithms ([explained here](https://machinelearningmastery.com/random-oversampling-and-undersampling-for-imbalanced-classification/)).

Random oversampling duplicates examples from the minority class in the dataset but can result in overfitting.

Random undersampling removes samples from the majority class but can result in the loss of important information.

Example implementation of SMOTE oversampling [here](https://beckernick.github.io/oversampling-modeling/).

In [221]:
# Good library for this is the imbalanced-learn python library
import imblearn

Feature selection: MRMR or Recursive Feature Elimination (RFE) or Boruta

You could consider feature selection a hyperparameter. Consequently, you can include it as part of a cross validation grid search to obtain optimum hyperparameters for the model in question.

https://medium.com/data-science-reporter/feature-selection-via-grid-search-in-supervised-models-4dc0c43d7ab1

Preproccessing:

- Clean data
- Add thyroid cancer
- Colum names
- Manual feature selection (data leakage - look at mutual information)


Pipeline:

Test-Train split

1. One-Hot encoding (not for random forests & some others)
2. Imputation
3. Feature scaling (not always)
4. Oversampling (not always)
5. Model selection
6. Cross validation grid search for hyperparameters and feature selection
7. Build Model
8.

Logging!